# Testing fixed angles with Carleton's code

In [ ]:
degree = 3 # degree of graph
rounds = 2 # number of rounds

# Import modules and packages
import numpy as np
import json
import os
import sys
sys.path[1:1] = [ "../_common/" ]
import common
import CC_solve_qaoa_long as sql 
import matplotlib.pyplot as plt

# Load the fixed angles from angles_regular_graphs.json taken from https://github.com/danlkv/fixed-angle-QAOA
with open(os.path.join("..", "_common", 'angles_regular_graphs.json'), 'r') as json_file:
    fixed_angles_orig = json.load(json_file) #'thetas_array', 'approx_ratio_list', 'num_qubits_list'
deg_3_p_2 = fixed_angles_orig[str(degree)][str(rounds)]
thetas_list = deg_3_p_2['beta'] + deg_3_p_2['gamma'] # betas are first followed by gammas
# thetas_list = [t for t in thetas_list]
for ind in range(rounds, 2 * rounds):
    thetas_list[ind] *= -1/2

In [ ]:

def ar_func(num_qubits = 4, rounds = 2, degree = 3, num_shots = 5000):
    # Retrieve graph edges and optimal solution
    unique_id = i = degree
    instance_filename = os.path.join( "..", "_common", 'instances',f"mc_{num_qubits:03d}_{i:03d}_000.txt")
    nodes, edges = common.read_maxcut_instance(instance_filename)
    opt, _ = common.read_maxcut_solution(instance_filename[:-4]+'.sol')

    G = (nodes, edges)
    func = sql.compute_expectation(G, num_shots)
    return - func(thetas_list)/opt


In [ ]:
num_qubits_list = list(range(4,17,2))
ar_list = [ar_func(num_qubits) for num_qubits in num_qubits_list]

In [ ]:
fig, axs = plt.subplots(1, 1)
suptitle = "Approximation Ratio for Fixed Angle Simulation\n rounds$={}$, degree=${}$ \n".format(rounds, degree)
angle_str = ""
for ind in range(rounds):
    angle_str += r"($\beta_{}={:.3f}$,".format(ind + 1, deg_3_p_2['beta'][ind])
    angle_str += r" $\gamma_{}={:.3f}$) ".format(ind + 1, deg_3_p_2['gamma'][ind]) + "\n"
angle_str = angle_str[:-2]
# and add the title to the plot
plt.suptitle(suptitle + angle_str)

# axs.plot(num_qubits_list, svs_list, marker='o', ls = '-', label = f"State Vector Simulation")
axs.plot(num_qubits_list, ar_list, marker='o', ls = '-', label = f"CC code QASM Simulation")
axs.axhline(y = deg_3_p_2['AR'], ls = 'dashed', label = f"Performance Guarantee")

axs.set_ylabel('Approximation Ratio')
axs.set_xlabel('Problem Size')
axs.grid()

axs.legend()

fig.tight_layout()
if not os.path.exists("__images"): os.mkdir("__images")
plt.savefig(os.path.join("__images", 'CC_fixed_angle.pdf'))
plt.savefig(os.path.join("__images", 'CC_fixed_angle.png'))